In [ ]:
import sys
import json
import logging
from pathlib import Path
from typing import List, Optional

# --- Configuration ---
CATALOG_FILE = Path("data/catalog.json")

# --- Task 5: Logging Setup ---
# Configure a unified logger for the application
logging.basicConfig(level=logging.INFO, 
                    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
                    handlers=[
                        logging.FileHandler("library_manager.log"),
                        logging.StreamHandler(sys.stdout) # Use stdout for console output
                    ])
logger = logging.getLogger("LibraryApp")


# ==============================================================================
# Task 1: Book Class
# ==============================================================================

class Book:
    """Represents a book in the library with a title, author, ISBN, and status."""
    
    STATUS_AVAILABLE = "available"
    STATUS_ISSUED = "issued"

    def __init__(self, title: str, author: str, isbn: str, status: str = STATUS_AVAILABLE):
        """Initializes a new Book object."""
        self.title = title
        self.author = author
        self.isbn = isbn
        # Basic status validation
        if status not in (self.STATUS_AVAILABLE, self.STATUS_ISSUED):
            self.status = self.STATUS_AVAILABLE
        else:
            self.status = status

    def __str__(self):
        """Returns a user-friendly string representation of the book."""
        return (f"Title: {self.title}, Author: {self.author}, ISBN: {self.isbn}, "
                f"Status: {self.status.capitalize()}")

    def to_dict(self):
        """Returns a dictionary representation of the book for JSON serialization."""
        return {
            "title": self.title,
            "author": self.author,
            "isbn": self.isbn,
            "status": self.status
        }

    def issue(self) -> bool:
        """Changes the book's status to issued if it's available."""
        if self.is_available():
            self.status = self.STATUS_ISSUED
            return True
        return False

    def return_book(self) -> bool:
        """Changes the book's status to available if it's issued."""
        if not self.is_available():
            self.status = self.STATUS_AVAILABLE
            return True
        return False

    def is_available(self) -> bool:
        """Checks if the book is currently available."""
        return self.status == self.STATUS_AVAILABLE


# ==============================================================================
# Task 2, 3, 5: Inventory Manager & Persistence
# ==============================================================================

class LibraryInventory:
    """Manages the collection of Book objects, including loading and saving to a file."""
    
    def __init__(self, file_path: Path):
        """Initializes the inventory and attempts to load data."""
        self.file_path = file_path
        self.books: List[Book] = []
        self._load_catalog()

    # --- Persistence Methods (Task 3 & 5) ---

    def _load_catalog(self):
        """Loads the book catalog from the JSON file."""
        logger.info(f"Attempting to load catalog from {self.file_path}")
        try:
            if not self.file_path.exists():
                logger.warning("Catalog file not found. Starting with an empty inventory.")
                return

            with open(self.file_path, 'r', encoding='utf-8') as f:
                data = json.load(f)
                
            # Convert dictionary data back into Book objects
            self.books = [
                Book(
                    item.get('title', 'Unknown Title'), 
                    item.get('author', 'Unknown Author'), 
                    item.get('isbn', '0000000000'), 
                    item.get('status', Book.STATUS_AVAILABLE)
                ) for item in data
            ]
            logger.info(f"Successfully loaded {len(self.books)} books.")

        except json.JSONDecodeError:
            # Task 5: Handle corrupted file
            logger.error("File is corrupted (JSON Decode Error). Inventory is empty.")
            self.books = []
        except Exception as e:
            # Task 5: Handle other potential file exceptions
            logger.error(f"An unexpected error occurred during loading: {e}", exc_info=True)
            self.books = []
        finally:
            logger.info("Catalog load attempt finished.")


    def save_catalog(self) -> bool:
        """Saves the current book catalog to the JSON file."""
        logger.info(f"Attempting to save catalog to {self.file_path}")
        try:
            data_to_save = [book.to_dict() for book in self.books]
            
            # Ensure the parent directory exists
            self.file_path.parent.mkdir(parents=True, exist_ok=True)
            
            with open(self.file_path, 'w', encoding='utf-8') as f:
                json.dump(data_to_save, f, indent=4)
            
            logger.info(f"Successfully saved {len(self.books)} books.")
            return True
        except Exception as e:
            logger.error(f"Error saving catalog: {e}", exc_info=True)
            return False

    # --- Inventory Management Methods (Task 2) ---

    def add_book(self, book: Book) -> bool:
        """Adds a book to the inventory if its ISBN is unique."""
        if self.search_by_isbn(book.isbn):
            logger.warning(f"Book with ISBN {book.isbn} already exists.")
            return False
        self.books.append(book)
        logger.info(f"Added book: {book.title}")
        return True

    def search_by_title(self, title: str) -> List[Book]:
        """Searches for books whose title contains the search string (case-insensitive)."""
        search_title = title.lower()
        return [book for book in self.books if search_title in book.title.lower()]

    def search_by_isbn(self, isbn: str) -> Optional[Book]:
        """Searches for a book by exact ISBN match."""
        for book in self.books:
            if book.isbn == isbn:
                return book
        return None

    def display_all(self):
        """Prints a list of all books in the inventory."""
        if not self.books:
            print("\n--- Inventory is Empty ---")
            return
        
        print(f"\n--- Current Inventory ({len(self.books)} Books) ---")
        # Sort books by title for a cleaner display
        sorted_books = sorted(self.books, key=lambda b: b.title)
        for i, book in enumerate(sorted_books, 1):
            print(f"[{i}] {book}")
        print("-----------------------------------")


# ==============================================================================
# Task 4 & 5: Menu-Driven CLI
# ==============================================================================

# --- Helper Functions for CLI ---

def get_user_input(prompt: str, validation_func=lambda x: x.strip()) -> Optional[str]:
    """Handles getting user input with basic validation and error handling (Task 5)."""
    while True:
        try:
            user_input = input(prompt).strip()
            if not user_input and validation_func != (lambda x: x.strip()):
                raise ValueError("Input cannot be empty.")
            
            validated_input = validation_func(user_input)
            if validated_input is None:
                # If custom validation returns None for failure, prompt again
                raise ValueError("Invalid format. Please follow the required input format.")
            return validated_input
            
        except EOFError:
            logger.error("EOF encountered. Exiting input routine.")
            return None
        except ValueError as e:
            print(f"Invalid input: {e}. Please try again.")
        except Exception as e:
            logger.error(f"An unexpected error occurred during input: {e}")
            return None

def validate_isbn(isbn: str) -> str:
    """Validation for ISBN (must be digits)."""
    if not isbn.isdigit():
        raise ValueError("ISBN must contain only digits (0-9).")
    return isbn

# --- CLI Command Functions ---

def add_book_cli(inventory: LibraryInventory):
    """Command to get book details from the user and add it to the inventory."""
    print("\n--- Add New Book ---")
    title = get_user_input("Enter Title: ")
    author = get_user_input("Enter Author: ")
    isbn = get_user_input("Enter ISBN (digits only): ", validate_isbn)

    if not all([title, author, isbn]):
        print("Operation cancelled or failed to get required input.")
        return

    # Check for existing book by ISBN
    if inventory.search_by_isbn(isbn):
        print(f"Error: A book with ISBN **{isbn}** already exists.")
        return

    try:
        new_book = Book(title, author, isbn)
        if inventory.add_book(new_book):
            print(f"\nSUCCESS: Book '**{title}**' added.")
            inventory.save_catalog()
        else:
            print(f"\nFAILURE: Could not add book '**{title}**'.")
    except Exception as e:
        logger.error(f"Error creating/adding book: {e}", exc_info=True)


def manage_status_cli(inventory: LibraryInventory, action: str):
    """General function to issue or return a book."""
    action_verb = "Issue" if action == "issue" else "Return"
    print(f"\n--- {action_verb} Book ---")
    isbn = get_user_input("Enter ISBN of the book: ", validate_isbn)
    
    if not isbn:
        print("Operation cancelled.")
        return

    book = inventory.search_by_isbn(isbn)

    if not book:
        print(f"ERROR: Book with ISBN **{isbn}** not found.")
        return

    try:
        if action == "issue":
            if book.issue():
                print(f"SUCCESS: Book '**{book.title}**' is now **ISSUED**.")
                inventory.save_catalog()
            else:
                print(f"FAILURE: Book '**{book.title}**' is already issued.")
        
        elif action == "return":
            if book.return_book():
                print(f"SUCCESS: Book '**{book.title}**' is now **AVAILABLE**.")
                inventory.save_catalog()
            else:
                print(f"FAILURE: Book '**{book.title}**' is already available (was never issued).")

    except Exception as e:
        logger.error(f"Error during {action} operation: {e}", exc_info=True)


def search_book_cli(inventory: LibraryInventory):
    """Command to search books by title or ISBN."""
    print("\n--- Search Book ---")
    print("Search by: (1) Title | (2) ISBN")
    
    # Input validation for menu choice
    choice = get_user_input("Enter choice (1 or 2): ", lambda x: x if x in ['1', '2'] else None)
    
    if choice == '1':
        search_term = get_user_input("Enter partial title to search: ")
        if search_term:
            results = inventory.search_by_title(search_term)
            if results:
                print(f"\n--- Search Results for Title containing '{search_term}' ({len(results)} found) ---")
                for i, book in enumerate(results, 1):
                    print(f"[{i}] {book}")
                print("-------------------------------------------------")
            else:
                print(f"No books found with title containing '**{search_term}**'.")
    
    elif choice == '2':
        search_term = get_user_input("Enter ISBN to search: ", validate_isbn)
        if search_term:
            result = inventory.search_by_isbn(search_term)
            if result:
                print(f"\n--- Search Result for ISBN **{search_term}** ---")
                print(result)
                print("----------------------------------------")
            else:
                print(f"Book with ISBN **{search_term}** not found.")
    else:
        print("Invalid search choice. Operation cancelled.")


# --- Main Application Logic ---

def main():
    """Main function to run the command-line interface."""
    print("📚 Starting Library Management System...")
    
    # Initialize the Inventory, which automatically attempts to load data (Task 3 & 5)
    inventory = LibraryInventory(CATALOG_FILE)

    while True:
        print("\n==================================")
        print("       LIBRARY MANAGEMENT MENU    ")
        print("==================================")
        print("1. Add New Book")
        print("2. Issue Book")
        print("3. Return Book")
        print("4. View All Books")
        print("5. Search Book")
        print("6. Exit and Save")
        print("----------------------------------")

        try:
            # Input validation for menu choice
            choice = get_user_input("Enter your choice (1-6): ", lambda x: x if x in ['1','2','3','4','5','6'] else None)

            if choice == '1':
                add_book_cli(inventory)
            elif choice == '2':
                manage_status_cli(inventory, "issue")
            elif choice == '3':
                manage_status_cli(inventory, "return")
            elif choice == '4':
                inventory.display_all()
            elif choice == '5':
                search_book_cli(inventory)
            elif choice == '6':
                logger.info("Exit command received. Attempting to save catalog...")
                inventory.save_catalog()
                print("\n✅ Catalog saved. Exiting application. Goodbye!")
                break
            else:
                # This branch should be unreachable due to `get_user_input` validation
                print("Invalid choice. Please enter a number between 1 and 6.")
        
        except KeyboardInterrupt:
            # Task 5: Handle graceful exit on Ctrl+C
            logger.info("Keyboard interrupt received. Attempting to save catalog...")
            inventory.save_catalog()
            print("\n\nExiting application. Goodbye!")
            break
        except Exception as e:
            logger.critical(f"A fatal error occurred: {e}", exc_info=True)
            print("\nA fatal error occurred. Check the **library_manager.log** file for details.")
            break

if __name__ == "__main__":
    main()

📚 Starting Library Management System...
2025-11-30 22:58:57,531 - LibraryApp - INFO - Attempting to load catalog from data\catalog.json
2025-11-30 22:58:57,537 - LibraryApp - WARNING - Catalog file not found. Starting with an empty inventory.
2025-11-30 22:58:57,538 - LibraryApp - INFO - Catalog load attempt finished.

       LIBRARY MANAGEMENT MENU    
1. Add New Book
2. Issue Book
3. Return Book
4. View All Books
5. Search Book
6. Exit and Save
----------------------------------


Enter your choice (1-6):  1



--- Add New Book ---


Enter Title:  snow
Enter Author:  snowiee
Enter ISBN (digits only):  23456


2025-11-30 22:59:42,901 - LibraryApp - INFO - Added book: snow

SUCCESS: Book '**snow**' added.
2025-11-30 22:59:42,903 - LibraryApp - INFO - Attempting to save catalog to data\catalog.json
2025-11-30 22:59:42,908 - LibraryApp - INFO - Successfully saved 1 books.

       LIBRARY MANAGEMENT MENU    
1. Add New Book
2. Issue Book
3. Return Book
4. View All Books
5. Search Book
6. Exit and Save
----------------------------------


...


--- Current Inventory (2 Books) ---
[1] Title: Haunt, Author: blue, ISBN: 23444, Status: Available
[2] Title: snow, Author: snowiee, ISBN: 23456, Status: Available
-----------------------------------

       LIBRARY MANAGEMENT MENU    
1. Add New Book
2. Issue Book
3. Return Book
4. View All Books
5. Search Book
6. Exit and Save
----------------------------------


Enter your choice (1-6):  5



--- Search Book ---
Search by: (1) Title | (2) ISBN


Enter choice (1 or 2):  23444


Invalid input: Invalid format. Please follow the required input format.. Please try again.
